# HU Extension                     Assignment 05           E63 Big Data Analytics
## Emmanuel Aboah
### Issued on: Feb 24, 2023                    Due on Saturday by 11:59 AM EST, March 4th, 2023

# Problem 1. 
Use VMWare (Workstation or Fusion) Player to create an Ubuntu22.04 Virtual Machine. If you already have a functioning VM, you are welcome to use it. Demonstrate that by using ssh you can connect from your host machine to your VM. If you are using an EC2 Instance in the AWS Cloud you can use that instance instead. When capturing screenshots of your ssh dialog, please include the date and time of your experiments. 

For my solutions I will be using Windows Subsystem Linux (WSL2) and running a single node cluster of kafka using docker-compose.

```bash
# Ubuntu version
(base) manny@LAPTOP-85L1BUVJ:~$ lsb_release -a
No LSB modules are available.
Distributor ID: Ubuntu
Description:    Ubuntu 18.04.6 LTS
Release:        18.04
Codename:       bionic

# Docker version
(base) manny@LAPTOP-85L1BUVJ:~$ docker version
Client: Docker Engine - Community
 Cloud integration: v1.0.29
 Version:           20.10.22
 API version:       1.41
 Go version:        go1.18.9
 Git commit:        3a2c30b
 Built:             Thu Dec 15 22:28:22 2022
 OS/Arch:           linux/amd64
 Context:           default
 Experimental:      true

Server: Docker Desktop
 Engine:
  Version:          20.10.22
  API version:      1.41 (minimum version 1.12)
  Go version:       go1.18.9
  Git commit:       42c8b31
  Built:            Thu Dec 15 22:26:14 2022
  OS/Arch:          linux/amd64
  Experimental:     false
 containerd:
  Version:          1.6.14
  GitCommit:        9ba4b250366a5ddde94bb7c9d1def331423aa323
 runc:
  Version:          1.1.4
  GitCommit:        v1.1.4-0-g5fd4c4d
 docker-init:
  Version:          0.19.0
  GitCommit:        de40ad0
```

# Problem 2. 
From http://kafka.apache.org, download and install single node Kafka on above VM. You are welcome to try the most recent version 3.4 or somewhat older 3.3.2. Install your Kafka for Scala 2.12.x. Demonstrate that you can start both Zookeeper and Kafka server. Demonstrate that you can start either of them and stop either of them. How will you find out whether Zookeeper is listening on port 2181 and Kafka server on port 9092? You might find appropriate Linux (Ubuntu) commands with Google’s help. 

Single node kafka cluster running in docker containers.
</br>

docker-compose.yaml

```yaml
version: "3.8"
networks:
  hw05-project:
    name: hw05-net
    driver: bridge
services:
  #Zookeeper container
  zookeeper:
    image: confluentinc/cp-zookeeper
    hostname: zookeeper
    container_name: zookeeper
    networks:
      - hw05-project
    ports:
      - "2181:2181"
    environment:
      ZOOKEEPER_CLIENT_PORT: 2181
      ZOOKEEPER_TICK_TIME: 2000

  #Kafka container node1
  broker1:
    image: confluentinc/cp-server
    hostname: broker1
    container_name: broker1
    healthcheck:
      test: nc -z localhost 9092
    networks:
      - hw05-project
    depends_on:
      - zookeeper
    ports:
      - "9092:9092"
    environment:
      KAFKA_BROKER_ID: 1
      KAFKA_ZOOKEEPER_CONNECT: 'zookeeper:2181'
      KAFKA_LISTENER_SECURITY_PROTOCOL_MAP: PLAINTEXT:PLAINTEXT,PLAINTEXT_HOST:PLAINTEXT
      KAFKA_INTER_BROKER_LISTENER_NAME: PLAINTEXT
      KAFKA_ADVERTISED_LISTENERS: PLAINTEXT://broker1:29092,PLAINTEXT_HOST://localhost:9092
      KAFKA_LISTENERS: PLAINTEXT://:29092,PLAINTEXT_HOST://:9092
      KAFKA_METRIC_REPORTERS: io.confluent.metrics.reporter.ConfluentMetricsReporter
      KAFKA_OFFSETS_TOPIC_REPLICATION_FACTOR: 1
      KAFKA_GROUP_INITIAL_REBALANCE_DELAY_MS: 0
      KAFKA_CONFLUENT_LICENSE_TOPIC_REPLICATION_FACTOR: 1
      KAFKA_CONFLUENT_BALANCER_TOPIC_REPLICATION_FACTOR: 1
      KAFKA_TRANSACTION_STATE_LOG_MIN_ISR: 1
      KAFKA_TRANSACTION_STATE_LOG_REPLICATION_FACTOR: 1
      KAFKA_JMX_PORT: 9101
      KAFKA_JMX_HOSTNAME: localhost
      KAFKA_CONFLUENT_SCHEMA_REGISTRY_URL: http://schema-registry:8081
      CONFLUENT_METRICS_REPORTER_BOOTSTRAP_SERVERS: broker1:29092
      CONFLUENT_METRICS_REPORTER_TOPIC_REPLICAS: 1
      CONFLUENT_METRICS_ENABLE: 'false'
      CONFLUENT_SUPPORT_CUSTOMER_ID: 'anonymous'
```

Start the local cluster.

```bash
(BDA2023) manny@LAPTOP-85L1BUVJ:~/dev/cscie-63/hw05/docker$ docker-compose up -d
[+] Running 15/15
 ⠿ broker1 Pulled                                                                                                                                                                                                     95.0s
   ⠿ 3550ee360766 Pull complete                                                                                                                                                                                       14.5s
   ⠿ e3455c6406eb Pull complete                                                                                                                                                                                       69.3s
   ⠿ 95cdaa6975d4 Pull complete                                                                                                                                                                                       69.9s
   ⠿ 3d3883ed71ab Pull complete                                                                                                                                                                                       70.2s
   ⠿ c5df82bd3053 Pull complete                                                                                                                                                                                       70.9s
   ⠿ 40036816fad0 Pull complete                                                                                                                                                                                       71.0s
   ⠿ f7e8fecc2b94 Pull complete                                                                                                                                                                                       71.1s
   ⠿ e80d13bd6d87 Pull complete                                                                                                                                                                                       71.2s
   ⠿ a11e73b0229d Pull complete                                                                                                                                                                                       71.3s
   ⠿ 56e56e5cfea4 Pull complete                                                                                                                                                                                       93.4s
   ⠿ cbda59915832 Pull complete                                                                                                                                                                                       93.5s
 ⠿ zookeeper Pulled                                                                                                                                                                                                   75.8s
   ⠿ d23edc08b858 Pull complete                                                                                                                                                                                       74.2s
   ⠿ 9b1cb94f7c7c Pull complete                                                                                                                                                                                       74.3s
[+] Running 0/1
[+] Running 3/3-net  Creating                                                                                                                                                                                          0.1s
 ⠿ Network hw05-net     Created                                                                                                                                                                                        0.2s
 ⠿ Container zookeeper  Started                                                                                                                                                                                        2.7s
 ⠿ Container broker1    Started 

 # Check cluster status
(BDA2023) manny@LAPTOP-85L1BUVJ:~/dev/cscie-63/hw05/docker$ docker ps
CONTAINER ID   IMAGE                       COMMAND                  CREATED         STATUS                   PORTS                                        NAMES
9a6166ddd9c8   confluentinc/cp-server      "/etc/confluent/dock…"   8 minutes ago   Up 8 minutes (healthy)   0.0.0.0:9092->9092/tcp                       broker1
f73de8275331   confluentinc/cp-zookeeper   "/etc/confluent/dock…"   8 minutes ago   Up 8 minutes             2888/tcp, 0.0.0.0:2181->2181/tcp, 3888/tcp   zookeeper

# Stop zookeeper
(BDA2023) manny@LAPTOP-85L1BUVJ:~/dev/cscie-63/hw05/docker$ docker stop zookeeper
zookeeper
(BDA2023) manny@LAPTOP-85L1BUVJ:~/dev/cscie-63/hw05/docker$ docker ps
CONTAINER ID   IMAGE                    COMMAND                  CREATED          STATUS                    PORTS                    NAMES
9a6166ddd9c8   confluentinc/cp-server   "/etc/confluent/dock…"   11 minutes ago   Up 11 minutes (healthy)   0.0.0.0:9092->9092/tcp   broker1

# Alternatively
(BDA2023) manny@LAPTOP-85L1BUVJ:~/dev/cscie-63/hw05/docker$ docker exec -it zookeeper bash
[appuser@zookeeper ~]$ /bin/zookeeper-server-stop
[appuser@zookeeper ~]$ (BDA2023) manny@LAPTOP-85L1BUVJ:~/dev/cscie-63/hw05/docker$ docker ps
CONTAINER ID   IMAGE                    COMMAND                  CREATED             STATUS                    PORTS                    NAMES
9a6166ddd9c8   confluentinc/cp-server   "/etc/confluent/dock…"   About an hour ago   Up 59 minutes (healthy)   0.0.0.0:9092->9092/tcp   broker1

# Start zookeeper running ports showing for server/broker and zookeeper
(BDA2023) manny@LAPTOP-85L1BUVJ:~/dev/cscie-63/hw05/docker$ docker-compose start zookeeper
[+] Running 1/1
 ⠿ Container zookeeper  Started                                                                                                                                                                                        0.4s
(BDA2023) manny@LAPTOP-85L1BUVJ:~/dev/cscie-63/hw05/docker$ docker-compose ps
NAME                IMAGE                       COMMAND                  SERVICE             CREATED             STATUS                    PORTS
broker1             confluentinc/cp-server      "/etc/confluent/dock…"   broker1             12 minutes ago      Up 12 minutes (healthy)   0.0.0.0:9092->9092/tcp
zookeeper           confluentinc/cp-zookeeper   "/etc/confluent/dock…"   zookeeper           12 minutes ago      Up 12 seconds             2888/tcp, 0.0.0.0:2181->2181/tcp, 3888/tcp
```

# Problem 3. 
Create two new topics. Name topics following your first name and the last names. If you are John Smith, one topic should be named John and the other Smith. List existing topics using console commands. Describe both topics using console commands.

```bash
# Create firstname topic
(BDA2023) manny@LAPTOP-85L1BUVJ:~/dev/cscie-63/hw05/docker$ docker exec -it broker1 /bin/kafka-topics --bootstrap-server broker1:29092 --create --topic Manny --partitions 1
Created topic Manny.

# Create topic lastname
(BDA2023) manny@LAPTOP-85L1BUVJ:~/dev/cscie-63/hw05/docker$ docker exec -it broker1 /bin/kafka-topics --bootstrap-server broker1:29092 --create --topic Aboah --partitions 1
Created topic Aboah.

# List topics
(BDA2023) manny@LAPTOP-85L1BUVJ:~/dev/cscie-63/hw05/docker$ docker exec -it broker1  /bin/kafka-topics --bootstrap-server broker1:29092 --list
Aboah
Manny
__consumer_offsets
_confluent-command
_confluent-metrics
_confluent-telemetry-metrics
_confluent_balancer_api_state
_confluent_balancer_broker_samples
_confluent_balancer_partition_samples
```

# Problem 4. 
Demonstrate that you can publish messages to Kafka and consume messages from Kafka using Kafka console tools. Do that twice. The first time produce and “save” 4 uncompressed messages. Always send messages in the form of key:value pairs.  Find partition or log where those messages were stored and try to read them using one of text editors.  Report on the location of those logs and the content you discovered.

```bash
# Uncompressed messages

# Producer
> docker exec  -it broker1 /bin/kafka-console-producer --bootstrap-server broker1:29092 --topic Manny
>Message 1: Hello 
>Message 2: World
>Message 3: Fun
>Message 4: Games  

# Consumer
> docker exec -it broker1 /bin/kafka-console-consumer --bootstrap-server broker1:29092 --topic Manny
Message 1: Hello
Message 2: World
Message 3: Fun
Message 4: Games

# Topic logs
(BDA2023) manny@LAPTOP-85L1BUVJ:~/dev/cscie-63/hw05/docker$ docker exec  -it broker1 /bin/kafka-run-class kafka.tools.DumpLogSegments --files /var/lib/kafka/data/Manny-0/00000000000000000000.log --print-data-log
Dumping /var/lib/kafka/data/Manny-0/00000000000000000000.log
Log starting offset: 0
baseOffset: 0 lastOffset: 0 count: 1 baseSequence: 0 lastSequence: 0 producerId: 0 producerEpoch: 0 partitionLeaderEpoch: 0 isTransactional: false isControl: false deleteHorizonMs: OptionalLong.empty position: 0 CreateTime: 1677813380594 size: 84 magic: 2 compresscodec: none crc: 3424044466 isvalid: true
| offset: 0 CreateTime: 1677813380594 keySize: -1 valueSize: 16 sequence: 0 headerKeys: [] payload: Message 1: Hello
baseOffset: 1 lastOffset: 1 count: 1 baseSequence: 1 lastSequence: 1 producerId: 0 producerEpoch: 0 partitionLeaderEpoch: 0 isTransactional: false isControl: false deleteHorizonMs: OptionalLong.empty position: 84 CreateTime: 1677813406214 size: 84 magic: 2 compresscodec: none crc: 2878688754 isvalid: true
| offset: 1 CreateTime: 1677813406214 keySize: -1 valueSize: 16 sequence: 1 headerKeys: [] payload: Message 2: World
baseOffset: 2 lastOffset: 2 count: 1 baseSequence: 2 lastSequence: 2 producerId: 0 producerEpoch: 0 partitionLeaderEpoch: 0 isTransactional: false isControl: false deleteHorizonMs: OptionalLong.empty position: 168 CreateTime: 1677813420440 size: 82 magic: 2 compresscodec: none crc: 2325924200 isvalid: true
| offset: 2 CreateTime: 1677813420440 keySize: -1 valueSize: 14 sequence: 2 headerKeys: [] payload: Message 3: Fun
baseOffset: 3 lastOffset: 3 count: 1 baseSequence: 3 lastSequence: 3 producerId: 0 producerEpoch: 0 partitionLeaderEpoch: 0 isTransactional: false isControl: false deleteHorizonMs: OptionalLong.empty position: 250 CreateTime: 1677813438626 size: 84 magic: 2 compresscodec: none crc: 3076656657 isvalid: true
| offset: 3 CreateTime: 1677813438626 keySize: -1 valueSize: 16 sequence: 3 headerKeys: [] payload: Message 4: Games 
```

# Problem 5. 
Repeat the tasks of Problem 4, this time saving messages in one of compressed forms. Please tell us how you made the broker compress the messages. Again, open the topic partition with a Vi or some other text editor and examine stored messages.

```bash
# Compressed message

# Create compressed message topic
(BDA2023) manny@LAPTOP-85L1BUVJ:~/dev/cscie-63$ docker exec -it broker1 /bin/kafka-topics --bootstrap-server broker1:29092 --create --topic manny-compressed-topic --config compression.type=gzip --replication-factor 1 --partitions 1
Created topic manny-compressed-topic.

# Producer - compressed
(BDA2023) manny@LAPTOP-85L1BUVJ:~/dev/cscie-63$ docker exec  -it broker1 /bin/kafka-console-producer --bootstrap-server broker1:29092 --topic manny-compressed-topic
>m1: This
>m2: is a 
>m3: compressed
>m3: message :)

# Consumer - compressed
(BDA2023) manny@LAPTOP-85L1BUVJ:~/dev/cscie-63$ docker exec -it broker1 /bin/kafka-console-consumer --bootstrap-server broker1:29092 --topic manny-compressed-topic
m1: This
m2: is a
m3: compressed
m3: message :)

# Topic logs
(BDA2023) manny@LAPTOP-85L1BUVJ:~/dev/cscie-63/hw05/docker$ docker exec  -it broker1 /bin/kafka-run-class kafka.tools.DumpLogSegments --files /var/lib/kafka/data/manny-compressed-topic-0/00000000000000000000.log --print-data-log
Dumping /var/lib/kafka/data/manny-compressed-topic-0/00000000000000000000.log
Log starting offset: 0
baseOffset: 0 lastOffset: 0 count: 1 baseSequence: 0 lastSequence: 0 producerId: 1 producerEpoch: 0 partitionLeaderEpoch: 0 isTransactional: false isControl: false deleteHorizonMs: OptionalLong.empty position: 0 CreateTime: 1677814327219 size: 96 magic: 2 compresscodec: gzip crc: 211065228 isvalid: true
| offset: 0 CreateTime: 1677814327219 keySize: -1 valueSize: 8 sequence: 0 headerKeys: [] payload: m1: This
baseOffset: 1 lastOffset: 1 count: 1 baseSequence: 1 lastSequence: 1 producerId: 1 producerEpoch: 0 partitionLeaderEpoch: 0 isTransactional: false isControl: false deleteHorizonMs: OptionalLong.empty position: 96 CreateTime: 1677814335441 size: 96 magic: 2 compresscodec: gzip crc: 3224285084 isvalid: true
| offset: 1 CreateTime: 1677814335441 keySize: -1 valueSize: 8 sequence: 1 headerKeys: [] payload: m2: is a
baseOffset: 2 lastOffset: 2 count: 1 baseSequence: 2 lastSequence: 2 producerId: 1 producerEpoch: 0 partitionLeaderEpoch: 0 isTransactional: false isControl: false deleteHorizonMs: OptionalLong.empty position: 192 CreateTime: 1677814343813 size: 102 magic: 2 compresscodec: gzip crc: 1550876762 isvalid: true
| offset: 2 CreateTime: 1677814343813 keySize: -1 valueSize: 14 sequence: 2 headerKeys: [] payload: m3: compressed
baseOffset: 3 lastOffset: 3 count: 1 baseSequence: 3 lastSequence: 3 producerId: 1 producerEpoch: 0 partitionLeaderEpoch: 0 isTransactional: false isControl: false deleteHorizonMs: OptionalLong.empty position: 294 CreateTime: 1677814352493 size: 102 magic: 2 compresscodec: gzip crc: 4116475984 isvalid: true
| offset: 3 CreateTime: 1677814352493 keySize: -1 valueSize: 14 sequence: 3 headerKeys: [] payload: m3: message :)
```

# Problem 6. 
Please visit Confluent Web site (https://www.confluent.io/ ) and locate the latest versions of Python code for a Kafka producer and Kafka consumer. Please provide us with the URL of the page where you located either code. If there are any differences in the formulation as compared to the code on slides 62 and 64 and 65 of the lecture notes, please report on the differences. Make two clients code taken from Confluent work. As a message to the topic, please use the JSON object described on slide 94 of the lecture notes. Your code should read a text file containing that JSON object. Submit working code and captured dialogues. You need to create the topic before you start the producer and the consumer.

References:
- [Python Producer](https://developer.confluent.io/get-started/python/#build-producer)
- [Python Consumer](https://developer.confluent.io/get-started/python/#build-consumer)
- [Config File](https://developer.confluent.io/get-started/python/#configuration)
- [Kafka Docker Setup](https://developer.confluent.io/get-started/python/#kafka-setup)

The main differences between the current confluent solution i am using here and the one from the slides are:
1. The app in the slides has only function definitions and will require a main method or driver class to exectue the functions.
2. The current confluent app uses environment config files to set kafka configurations and properties.
3. The current confluent app iterates over two dictionaries and produces messages from them as opposed to the bite encoded (kafkatest *20) string in the slides.

Configuration file. <br>

kafka_env.ini

```bash
[default]
bootstrap.servers=localhost:9092

[consumer]
group.id=python_group_1

# 'auto.offset.reset=earliest' to start reading from the beginning of
# the topic if no committed offsets exist.
auto.offset.reset=latest
```

Shoppers Json file.

```json
{
    "event": "SHOPPER_VIEWED_PRODUCT",
    "shopper": {
        "id": "123",
        "name": "Jane",
        "ipAddress": "70.46.123.145"
    },
    "product": {
        "sku": "aapl-001",
        "name": "iPad"
    },
    "timestamp": "2018-10-15T12:01:35Z"
}
```

Python Producer

```python
import json
from argparse import ArgumentParser, FileType
from configparser import ConfigParser
from confluent_kafka import Producer
from uuid import uuid4

if __name__ == '__main__':
    # Parse the command line.
    parser = ArgumentParser()
    parser.add_argument('config_file', type=FileType('r'))
    args = parser.parse_args()

    # Parse the configuration.
    # See https://github.com/edenhill/librdkafka/blob/master/CONFIGURATION.md
    config_parser = ConfigParser()
    config_parser.read_file(args.config_file)
    config = dict(config_parser['default'])

    # Create Producer instance
    producer = Producer(config)

    # Optional per-message delivery callback (triggered by poll() or flush())
    # when a message has been successfully delivered or permanently
    # failed delivery (after retries).
    def delivery_callback(err, msg):
        if err:
            print('ERROR: Message failed delivery: {}'.format(err))
        else:
            print("Produced event to topic {topic}: key = {key:12} value = {value:12}".format(
                topic=msg.topic(), key=msg.key().decode('utf-8'), value=msg.value().decode('utf-8')))

    # Produce data by selecting random values from these lists.
    topic = "shopper"
    json_data = open("shopper_data.json")
    data = str(json.load(json_data)).encode()

    try:
        producer.produce(topic=topic, key=str(uuid4()), value=data, callback=delivery_callback)
        # Block until the messages are sent.
        producer.poll(10000)
        producer.flush()
    except Exception as ex:
        print("Exception happend: ", ex)
```

Output

```bash
(BDA2023) manny@LAPTOP-85L1BUVJ:~/dev/cscie-63/hw05/python$ python producer.py kafa_env.ini
Produced event to topic shopper: key = 6ae07532-0108-4aaa-ae56-e90b5d4bcddb value = {'event': 'SHOPPER_VIEWED_PRODUCT', 'shopper': {'id': '123', 'name': 'Jane', 'ipAddress': '70.46.123.145'}, 'product': {'sku': 'aapl-001', 'name': 'iPad'}, 'timestamp': '2018-10-15T12:01:35Z'}
```

Consumer App

```python
from argparse import ArgumentParser, FileType
from configparser import ConfigParser
from confluent_kafka import Consumer, OFFSET_BEGINNING

if __name__ == '__main__':
    # Parse the command line.
    parser = ArgumentParser()
    parser.add_argument('config_file', type=FileType('r'))
    parser.add_argument('--reset', action='store_true')
    args = parser.parse_args()

    # Parse the configuration.
    # See https://github.com/edenhill/librdkafka/blob/master/CONFIGURATION.md
    config_parser = ConfigParser()
    config_parser.read_file(args.config_file)
    config = dict(config_parser['default'])
    config.update(config_parser['consumer'])

    # Create Consumer instance
    consumer = Consumer(config)

    # Set up a callback to handle the '--reset' flag.
    def reset_offset(consumer, partitions):
        if args.reset:
            for p in partitions:
                p.offset = OFFSET_BEGINNING
            consumer.assign(partitions)

    # Subscribe to topic
    topic = "shopper"
    consumer.subscribe([topic], on_assign=reset_offset)

    # Poll for new messages from Kafka and print them.
    try:
        while True:
            msg = consumer.poll(1.0)
            if msg is None:
                # Initial message consumption may take up to
                # `session.timeout.ms` for the consumer group to
                # rebalance and start consuming
                print("Waiting...")
            elif msg.error():
                print("ERROR: %s".format(msg.error()))
            else:
                # Extract the (optional) key and value, and print.

                print("Consumed event from topic {topic}: key = {key} value = {value}".format(
                    topic=msg.topic(), key=msg.key().decode('utf-8'), value=msg.value().decode('utf-8')))
    except KeyboardInterrupt:
        pass
    finally:
        # Leave group and commit final offsets
        consumer.close()
```

Output

```bash
(BDA2023) manny@LAPTOP-85L1BUVJ:~/dev/cscie-63/hw05/python$ python consumer.py kafa_env.ini
Consumed event from topic shopper: key = 6ae07532-0108-4aaa-ae56-e90b5d4bcddb value = {'event': 'SHOPPER_VIEWED_PRODUCT', 'shopper': {'id': '123', 'name': 'Jane', 'ipAddress': '70.46.123.145'}, 'product': {'sku': 'aapl-001', 'name': 'iPad'}, 'timestamp': '2018-10-15T12:01:35Z'}
```